In [43]:
import re
with open('input.txt', 'r') as f:
    input = f.read()
input = input.splitlines()
regex = re.compile(r'Blueprint (?P<blueprint_id>\d+): Each ore robot costs (?P<ore_robot_ore_cost>\d+) ore. Each clay robot costs (?P<clay_robot_ore_cost>\d+) ore. Each obsidian robot costs (?P<obsidian_robot_ore_cost>\d+) ore and (?P<obsidian_robot_clay_cost>\d+) clay. Each geode robot costs (?P<geode_robot_ore_cost>\d+) ore and (?P<geode_robot_obsidian_cost>\d+) obsidian.')
input = [regex.match(x).groupdict() for x in input]
input = [dict([a, int(x)] for a, x in b.items()) for b in input]

for blueprint in input:
    minute = 1
    bag = {
        "ore_robot": 1,
        "clay_robot": 0,
        "obsidian_robot": 0,
        "geode_robot": 0,
        "geode": 0,
        "ore": 0, 
        "clay": 0,
        "obsidian": 0,
    }
    while(minute < 25):
        # print(blueprint)
        # Spend Phase
        new_robots = [0, 0, 0, 0]
        test_obsidian = bag["obsidian"] / blueprint["geode_robot_obsidian_cost"]
        test_ore = bag["ore"] / blueprint["geode_robot_ore_cost"]
        geode_preference = test_obsidian + test_ore - 1
        if(blueprint["geode_robot_obsidian_cost"] <= bag["obsidian"] and blueprint["geode_robot_ore_cost"] <= bag["ore"] and geode_preference > 0):
            test_obsidian = bag["obsidian"] // blueprint["geode_robot_obsidian_cost"]
            test_ore = bag["ore"] // blueprint["geode_robot_ore_cost"]
            test_min = min(test_obsidian, test_ore)
            bag["obsidian"] -= test_min * blueprint["geode_robot_obsidian_cost"]
            bag["ore"] -= test_min * blueprint["geode_robot_ore_cost"]
            new_robots[0] = test_min
        test_clay = bag["clay"] / blueprint["obsidian_robot_clay_cost"]
        test_ore = bag["ore"] / blueprint["obsidian_robot_ore_cost"]
        obsidian_preference = test_clay + test_ore - geode_preference
        if(blueprint["obsidian_robot_clay_cost"] <= bag["clay"] and blueprint["obsidian_robot_ore_cost"] <= bag["ore"] and obsidian_preference > 0):
            test_clay = bag["clay"] // blueprint["obsidian_robot_clay_cost"]
            test_ore = bag["ore"] // blueprint["obsidian_robot_ore_cost"]
            test_min = min(test_clay, test_ore)
            bag["clay"] -= test_min * blueprint["obsidian_robot_clay_cost"]
            bag["ore"] -= test_min * blueprint["obsidian_robot_ore_cost"]
            new_robots[1] = test_min
        test_ore = bag["ore"] / blueprint["clay_robot_ore_cost"]
        clay_preference = test_ore - obsidian_preference
        if(blueprint["clay_robot_ore_cost"] <= bag["ore"] and clay_preference > 0):
            test_ore = bag["ore"] // blueprint["clay_robot_ore_cost"]
            bag["ore"] -= test_ore * blueprint["clay_robot_ore_cost"]
            new_robots[2] = test_ore
        test_ore = bag["ore"] / blueprint["ore_robot_ore_cost"]
        ore_preference = test_ore - clay_preference
        if(blueprint["ore_robot_ore_cost"] <= bag["ore"] and ore_preference > 0):
            test_ore = bag["ore"] // blueprint["ore_robot_ore_cost"]
            bag["ore"] -= test_ore * blueprint["ore_robot_ore_cost"]
            new_robots[3] = test_ore
        print(geode_preference, obsidian_preference, clay_preference, ore_preference, new_robots)
        # Collect Phase
        for item in bag:
            if("robot" in item):
                bag[item.replace("_robot", "")] += 1 * bag[item]
        bag["geode_robot"] += new_robots[0]
        bag["obsidian_robot"] += new_robots[1]
        bag["clay_robot"] += new_robots[2]
        bag["ore_robot"] += new_robots[3]
        print(bag, minute)
        minute += 1


-1.0 1.0 -1.0 1.0 [0, 0, 0, 0]
{'ore_robot': 1, 'clay_robot': 0, 'obsidian_robot': 0, 'geode_robot': 0, 'geode': 0, 'ore': 1, 'clay': 0, 'obsidian': 0} 1
-0.5 0.8333333333333333 -0.33333333333333326 0.5833333333333333 [0, 0, 0, 0]
{'ore_robot': 1, 'clay_robot': 0, 'obsidian_robot': 0, 'geode_robot': 0, 'geode': 0, 'ore': 2, 'clay': 0, 'obsidian': 0} 2
0.0 0.6666666666666666 0.33333333333333337 -0.33333333333333337 [0, 0, 1, 0]
{'ore_robot': 1, 'clay_robot': 1, 'obsidian_robot': 0, 'geode_robot': 0, 'geode': 0, 'ore': 1, 'clay': 0, 'obsidian': 0} 3
-0.5 0.8333333333333333 -0.33333333333333326 0.5833333333333333 [0, 0, 0, 0]
{'ore_robot': 1, 'clay_robot': 1, 'obsidian_robot': 0, 'geode_robot': 0, 'geode': 0, 'ore': 2, 'clay': 1, 'obsidian': 0} 4
0.0 0.738095238095238 0.261904761904762 -0.261904761904762 [0, 0, 1, 0]
{'ore_robot': 1, 'clay_robot': 2, 'obsidian_robot': 0, 'geode_robot': 0, 'geode': 0, 'ore': 1, 'clay': 2, 'obsidian': 0} 5
-0.5 0.9761904761904762 -0.47619047619047616 0.7261